In [ ]:
import pandas as pd
import numpy as np
import joblib
from sklearn.preprocessing import StandardScaler
from sklearn.cluster import KMeans
from sklearn.metrics.pairwise import cosine_similarity
import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
df = pd.read_csv('/content/sample_data/spotify_tracks_dataset.csv')
print("Loaded rows,cols:", df.shape)
print("Columns:", df.columns.tolist()[:40])
df

Loaded rows,cols: (114000, 21)
Columns: ['Unnamed: 0', 'track_id', 'artists', 'album_name', 'track_name', 'popularity', 'duration_ms', 'explicit', 'danceability', 'energy', 'key', 'loudness', 'mode', 'speechiness', 'acousticness', 'instrumentalness', 'liveness', 'valence', 'tempo', 'time_signature', 'track_genre']


,Unnamed: 0,track_id,artists,album_name,track_name,popularity,duration_ms,explicit,danceability,energy,...,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,time_signature,track_genre
0,0,5SuOikwiRyPMVoIQDJUgSV,Gen Hoshino,Comedy,Comedy,73,230666,False,0.676,0.4610,...,-6.746,0,0.1430,0.0322,0.000001,0.3580,0.7150,87.917,4,acoustic
1,1,4qPNDBW1i3p13qLCt0Ki3A,Ben Woodward,Ghost (Acoustic),Ghost - Acoustic,55,149610,False,0.420,0.1660,...,-17.235,1,0.0763,0.9240,0.000006,0.1010,0.2670,77.489,4,acoustic
2,2,1iJBSr7s7jYXzM8EGcbK5b,Ingrid Michaelson;ZAYN,To Begin Again,To Begin Again,57,210826,False,0.438,0.3590,...,-9.734,1,0.0557,0.2100,0.000000,0.1170,0.1200,76.332,4,acoustic
3,3,6lfxq3CG4xtTiEg7opyCyx,Kina Grannis,Crazy Rich Asians (Original Motion Picture Sou...,Can't Help Falling In Love,71,201933,False,0.266,0.0596,...,-18.515,1,0.0363,0.9050,0.000071,0.1320,0.1430,181.740,3,acoustic
4,4,5vjLSffimiIP26QG5WcN2K,Chord Overstreet,Hold On,Hold On,82,198853,False,0.618,0.4430,...,-9.681,1,0.0526,0.4690,0.000000,0.0829,0.1670,119.949,4,acoustic
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
113995,113995,2C3TZjDRiAzdyViavDJ217,Rainy Lullaby,#mindfulness - Soft Rain for Mindful Meditatio...,Sleep My Little Boy,21,384999,False,0.172,0.2350,...,-16.393,1,0.0422,0.6400,0.928000,0.0863,0.0339,125.995,5,world-music
113996,113996,1hIz5L4IB9hN3WRYPOCGPw,Rainy Lullaby,#mindfulness - Soft Rain for Mindful Meditatio...,Water Into Light,22,385000,False,0.174,0.1170,...,-18.318,0,0.0401,0.9940,0.976000,0.1050,0.0350,85.239,4,world-music
113997,113997,6x8ZfSoqDjuNa5SVP5QjvX,Cesária Evora,Best Of,Miss Perfumado,22,271466,False,0.629,0.3290,...,-10.895,0,0.0420,0.8670,0.000000,0.0839,0.7430,132.378,4,world-music
113998,113998,2e6sXL2bYv4bSz6VTdnfLs,Michael W. Smith,Change Your World,Friends,41,283893,False,0.587,0.5060,...,-10.889,1,0.0297,0.3810,0.000000,0.2700,0.4130,135.960,4,world-music


In [ ]:
# Normalize column names to expected ones (safe mapping)
col_map = {}
if 'track_id' in df.columns and 'id' not in df.columns:
    col_map['track_id'] = 'id'
if 'track_name' in df.columns and 'name' not in df.columns:
    col_map['track_name'] = 'name'
if 'artists' in df.columns and 'artist' not in df.columns:
    col_map['artists'] = 'artist'
if 'track_genre' in df.columns and 'genre' not in df.columns:
    col_map['track_genre'] = 'genre'
df.rename(columns=col_map, inplace=True)

In [ ]:
# Create track_url column if not present
if 'track_url' not in df.columns:
    if 'id' in df.columns:
        df['track_url'] = "https://open.spotify.com/track/" + df['id'].astype(str)
    elif 'uri' in df.columns:  # sometimes 'uri' exists: spotify:track:xxxx
        df['track_url'] = df['uri'].apply(lambda x: "https://open.spotify.com/track/" + x.split(':')[-1] if isinstance(x,str) else None)
    else:
        df['track_url'] = None

In [ ]:
# Ensure basic columns exist
expected_cols = ['name','artist','track_url','valence','energy','danceability','tempo','loudness','acousticness','speechiness','instrumentalness','liveness']
print("Has expected columns:", {c: (c in df.columns) for c in expected_cols})

Has expected columns: {'name': True, 'artist': True, 'track_url': True, 'valence': True, 'energy': True, 'danceability': True, 'tempo': True, 'loudness': True, 'acousticness': True, 'speechiness': True, 'instrumentalness': True, 'liveness': True}


In [ ]:
# Keep rows with id/name and valence+energy (we need those)
keep_cols = ['id','name','artist','track_url','valence','energy','danceability','tempo','loudness','acousticness','speechiness','instrumentalness','liveness','genre']
# select columns that do exist
present = [c for c in keep_cols if c in df.columns]
df = df[present].copy()

In [ ]:
# Convert numeric features to numeric and drop rows with nulls in core features
numeric_features = ['danceability','energy','valence','tempo','loudness','acousticness','speechiness','instrumentalness','liveness']
numeric_present = [c for c in numeric_features if c in df.columns]
for c in numeric_present:
    df[c] = pd.to_numeric(df[c], errors='coerce')

In [ ]:
df = df.dropna(subset=['valence','energy'] + numeric_present).reset_index(drop=True)
print("After cleaning rows,cols:", df.shape)
df.head(3)

After cleaning rows,cols: (114000, 14)


,id,name,artist,track_url,valence,energy,danceability,tempo,loudness,acousticness,speechiness,instrumentalness,liveness,genre
0,5SuOikwiRyPMVoIQDJUgSV,Comedy,Gen Hoshino,https://open.spotify.com/track/5SuOikwiRyPMVoI...,0.715,0.461,0.676,87.917,-6.746,0.0322,0.1430,0.000001,0.358,acoustic
1,4qPNDBW1i3p13qLCt0Ki3A,Ghost - Acoustic,Ben Woodward,https://open.spotify.com/track/4qPNDBW1i3p13qL...,0.267,0.166,0.420,77.489,-17.235,0.9240,0.0763,0.000006,0.101,acoustic
2,1iJBSr7s7jYXzM8EGcbK5b,To Begin Again,Ingrid Michaelson;ZAYN,https://open.spotify.com/track/1iJBSr7s7jYXzM8...,0.120,0.359,0.438,76.332,-9.734,0.2100,0.0557,0.000000,0.117,acoustic


Scale features & save scaler

In [ ]:
from sklearn.preprocessing import MinMaxScaler
# Select feature set for clustering & similarity (order matters)
feature_cols = numeric_present.copy()  # as found above
print("Using features:", feature_cols)

Using features: ['danceability', 'energy', 'valence', 'tempo', 'loudness', 'acousticness', 'speechiness', 'instrumentalness', 'liveness']


In [ ]:
# Scale with StandardScaler or MinMax (I prefer Standard for distance metrics)
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
X = scaler.fit_transform(df[feature_cols].values)
joblib.dump(scaler, "/content/scaler.joblib")
print("Saved scaler")

Saved scaler


KMeans clustering (discover mood clusters)

In [ ]:
# Choose k (7)
k = 7
kmeans = KMeans(n_clusters=k, random_state=42, n_init=20)
labels = kmeans.fit_predict(X)
df['cluster'] = labels
joblib.dump(kmeans, "/content/kmeans.joblib")
print("KMeans done. Cluster sizes:")
print(df['cluster'].value_counts())

KMeans done. Cluster sizes:
cluster
4    36561
0    26436
1    22975
3    11731
2     7789
5     7392
6     1116
Name: count, dtype: int64


In [ ]:
# Inspect centroids (in original scaled feature space -> inverse scaling for human readable)
centroids = kmeans.cluster_centers_
# To view centroids in original feature space:
centroids_orig = scaler.inverse_transform(centroids)
centroids_df = pd.DataFrame(centroids_orig, columns=feature_cols)
centroids_df['cluster'] = centroids_df.index
display(centroids_df.round(3))

,danceability,energy,valence,tempo,loudness,acousticness,speechiness,instrumentalness,liveness,cluster
0,0.472,0.814,0.372,139.785,-5.334,0.075,0.083,0.033,0.192,0
1,0.531,0.376,0.392,113.115,-10.640,0.679,0.053,0.029,0.158,1
2,0.524,0.754,0.513,123.535,-6.980,0.283,0.087,0.065,0.737,2
3,0.587,0.741,0.340,126.868,-8.414,0.112,0.071,0.790,0.170,3
4,0.705,0.722,0.693,117.733,-6.393,0.208,0.093,0.018,0.164,4
5,0.346,0.169,0.185,103.044,-21.219,0.865,0.051,0.786,0.163,5
6,0.576,0.666,0.446,101.898,-11.173,0.728,0.828,0.010,0.643,6


Automatic cluster → mood labeling (heuristic)

In [ ]:
display(centroids_df[['valence', 'energy']])

,valence,energy
0,0.372335,0.814344
1,0.392165,0.376321
2,0.513246,0.753723
3,0.340494,0.740730
4,0.692849,0.721594
5,0.185107,0.169191
6,0.445897,0.666241


In [ ]:
#  FINAL MANUAL 7-EMOTION MAPPING FOR CLUSTERS

cluster_to_mood = {
    0: "angry",        # high energy, low valence
    1: "melancholic",  # low valence, mid-low energy
    2: "energetic",    # high energy, mid valence
    3: "happy",        # high energy (party/upbeat)
    4: "calm",         # high valence, smooth energy
    5: "sad",          # lowest valence+energy
    6: "romantic"      # emotional, mid-high energy
}

def centroid_to_mood(row):
    cid = row.name
    return cluster_to_mood.get(cid, "neutral")

In [ ]:
mood_to_cluster = {m: c for c, m in cluster_to_mood.items()}
print("Cluster->mood:", cluster_to_mood)
print("Mood->cluster:", mood_to_cluster)

Cluster->mood: {0: 'angry', 1: 'melancholic', 2: 'energetic', 3: 'happy', 4: 'calm', 5: 'sad', 6: 'romantic'}
Mood->cluster: {'angry': 0, 'melancholic': 1, 'energetic': 2, 'happy': 3, 'calm': 4, 'sad': 5, 'romantic': 6}


In [ ]:
# add mood labels to df
df['mood'] = df['cluster'].map(cluster_to_mood)
df['mood'].value_counts()

,count
mood,
calm,36561
angry,26436
melancholic,22975
happy,11731
energetic,7789
sad,7392
romantic,1116


Recommender functions (returns desired columns)

In [ ]:
# Feature matrix for similarity (scaled)
feature_matrix = X  # aligned to df rows

# id->index mapping
if 'id' in df.columns:
    id_to_idx = {str(r['id']): i for i, r in df[['id']].iterrows()}
else:
    id_to_idx = {i: i for i in df.index}

In [ ]:
# helper to get top N similar rows for a given vector
def top_n_from_vector(vec_scaled, top_n=10):
    sims = cosine_similarity(vec_scaled.reshape(1, -1), feature_matrix)[0]
    top_idx = np.argsort(sims)[::-1][:top_n]
    res = df.iloc[top_idx].copy()
    res = res.assign(similarity=sims[top_idx])

    # Build output DataFrame safely (no SettingWithCopyWarning)
    out_cols = ['track_url', 'artist', 'song_name', 'mood', 'similarity']
    out = pd.DataFrame(columns=out_cols)

    # fill columns safely
    if 'track_url' in res.columns:
        out.loc[:, 'track_url'] = res['track_url'].values
    else:
        out.loc[:, 'track_url'] = None

    if 'artist' in res.columns:
        out.loc[:, 'artist'] = res['artist'].values
    elif 'artists' in res.columns:
        out.loc[:, 'artist'] = res['artists'].values
    else:
        out.loc[:, 'artist'] = None

    if 'name' in res.columns:
        out.loc[:, 'song_name'] = res['name'].values
    elif 'track_name' in res.columns:
        out.loc[:, 'song_name'] = res['track_name'].values
    else:
        out.loc[:, 'song_name'] = None

    out.loc[:, 'mood'] = res['mood'].values
    out.loc[:, 'similarity'] = res['similarity'].values

    return out.reset_index(drop=True)

In [ ]:
# Recommend by track id
def recommend_by_track(track_id, top_n=10):
    tid = str(track_id)
    if tid not in id_to_idx:
        raise ValueError("track_id not found")
    idx = id_to_idx[tid]
    vec = feature_matrix[idx]
    return top_n_from_vector(vec, top_n=top_n)

In [ ]:
# Recommend by mood (closest to mood centroid)
def recommend_by_mood(mood_label, top_n=15):
    if mood_label not in mood_to_cluster:
        raise ValueError("unknown mood")
    cluster_id = mood_to_cluster[mood_label]
    # compute centroid of cluster in scaled space (mean of feature_matrix rows for that cluster)
    idxs = df.index[df['cluster'] == cluster_id].tolist()
    if len(idxs) == 0:
        return pd.DataFrame(columns=['track_url','artist','song_name','mood','similarity'])
    mood_center = feature_matrix[idxs].mean(axis=0)
    return top_n_from_vector(mood_center, top_n=top_n)

Quick interactive tests

In [ ]:
# show sample rows
display(df[['id','name','artist','track_url','mood']].sample(5))

# Example: recommend by mood
print("\nTop recommendations for 'energetic':")
try:
    recs = recommend_by_mood('energetic', top_n=10)
    display(recs)
except Exception as e:
    print("Error:", e)

,id,name,artist,track_url,mood
102356,2r5HFOAuvPhyo0qvY32Dlo,Ek Tukda Dhoop,Raghav Chaitanya,https://open.spotify.com/track/2r5HFOAuvPhyo0q...,melancholic
26446,2eKcH1Vww6L6ZAR76RIUfy,Todo el Mundo a Natación,Doctora Juguetes;Felpita;Lambie;Chica Surfer,https://open.spotify.com/track/2eKcH1Vww6L6ZAR...,calm
18533,10esld81LEYoPMKGzHsXm3,The Gross Pillow,Chad Daniels,https://open.spotify.com/track/10esld81LEYoPMK...,romantic
20927,2IURcEXwXgzREzWkr2foxQ,REMEDY,Alesso,https://open.spotify.com/track/2IURcEXwXgzREzW...,calm
46522,44yh6K9jMexBzeAV64Ccyi,Fucking Maximaal,Neophyte;Never Surrender,https://open.spotify.com/track/44yh6K9jMexBzeA...,angry


Top recommendations for 'energetic':


,track_url,artist,song_name,mood,similarity
0,https://open.spotify.com/track/719Dewg7OFXZEmA...,Los Fabulosos Cadillacs,Vasos Vacios - Versión Remasterizada (Live),energetic,0.993963
1,https://open.spotify.com/track/6GkUCsN1tEH24jo...,Jorge & Mateus,Do Brasil A Argentina (Tô Indo Te Buscar) - Ao...,energetic,0.992582
2,https://open.spotify.com/track/0nTXrWKh74sULHO...,Paulo Rogerio,Minha Casa e Eu - Ao Vivo,energetic,0.991835
3,https://open.spotify.com/track/6JMRXqVpDD1Xu2l...,O Rappa,Rodo cotidiano - Ao vivo,energetic,0.983604
4,https://open.spotify.com/track/6JMRXqVpDD1Xu2l...,O Rappa,Rodo cotidiano - Ao vivo,energetic,0.983604
5,https://open.spotify.com/track/6JMRXqVpDD1Xu2l...,O Rappa,Rodo cotidiano - Ao vivo,energetic,0.983604
6,https://open.spotify.com/track/6JMRXqVpDD1Xu2l...,O Rappa,Rodo cotidiano - Ao vivo,energetic,0.983604
7,https://open.spotify.com/track/6JMRXqVpDD1Xu2l...,O Rappa,Rodo cotidiano - Ao vivo,energetic,0.983604
8,https://open.spotify.com/track/1W81Hac4dJXLsJQ...,Diogo Nogueira;Chico Buarque;Ivan Lins;Hamilto...,"Sou Eu - Ao Vivo Em Vivo Rio,Brasil/2010",energetic,0.98305
9,https://open.spotify.com/track/7qY7qxZEyLcxxwX...,Ratones Paranoicos,Lo que doy - Vivo,energetic,0.982644


In [ ]:
# Example: recommend by seed track (first row)
seed_id = str(df.loc[0,'id'])
print("Seed:", df.loc[0,['name','artist','mood']].to_dict())
print(recommend_by_track(seed_id, top_n=8))

Seed: {'name': 'Comedy', 'artist': 'Gen Hoshino', 'mood': 'calm'}
                                           track_url            artist  \
0  https://open.spotify.com/track/5SuOikwiRyPMVoI...       Gen Hoshino   
1  https://open.spotify.com/track/5SuOikwiRyPMVoI...       Gen Hoshino   
2  https://open.spotify.com/track/5SuOikwiRyPMVoI...       Gen Hoshino   
3  https://open.spotify.com/track/5SuOikwiRyPMVoI...       Gen Hoshino   
4  https://open.spotify.com/track/6aBFjYfhZrgJtxi...          Gramatik   
5  https://open.spotify.com/track/1aR6KEUmuCvrI5Q...  Charlie Feathers   
6  https://open.spotify.com/track/3IORLAm07yxo4T7...        Jason Mraz   
7  https://open.spotify.com/track/4sZc1lIErPY5Vbc...           Danakil   

                            song_name  mood similarity  
0                              Comedy  calm        1.0  
1                              Comedy  calm        1.0  
2                              Comedy  calm        1.0  
3                              Comedy  

In [ ]:
!pip install -q transformers torch
from transformers import pipeline

classifier = pipeline(
    "text-classification",
    model="j-hartmann/emotion-english-distilroberta-base",
    return_all_scores=True
)

Device set to use cpu
/usr/local/lib/python3.12/dist-packages/transformers/pipelines/text_classification.py:111: UserWarning: `return_all_scores` is now deprecated,  if want a similar functionality use `top_k=None` instead of `return_all_scores=True` or `top_k=1` instead of `return_all_scores=False`.
  warnings.warn(


In [ ]:
def detect_mood_from_text(text):
    text = str(text).lower().strip()

    # Keyword-based detection FIRST
    keyword_map = {
    # Energetic
    "energetic": "energetic",
    "excited": "energetic",
    "hype": "energetic",
    "motivated": "energetic",
    "pumped": "energetic",

    # Calm / Relax
    "relax": "calm",
    "relaxing": "calm",
    "relaxed": "calm",
    "calm": "calm",
    "peaceful": "calm",
    "soothing": "calm",
    "chill": "calm",
    "rest": "calm",
    "tired": "calm",

    # Romantic
    "romantic": "romantic",
    "love": "romantic",
    "affection": "romantic",
    "crush": "romantic",

    # Sad
    "sad": "sad",
    "depressed": "sad",
    "cry": "sad",
    "heartbroken": "sad",
    "lonely": "sad",

    # Angry
    "angry": "angry",
    "mad": "angry",
    "irritated": "angry",
    "annoyed": "angry",

    # Melancholic
    "nostalgic": "melancholic",
    "nostalgia": "melancholic",
    "melancholic": "melancholic",
    "thinking": "melancholic"
}

    # Direct keyword match
    for key, mood in keyword_map.items():
        if key in text:
            return mood

    # Use HuggingFace emotional model
    scores = classifier(text)[0]  # list of dicts
    best = max(scores, key=lambda x: x['score'])
    label = best['label'].lower()

    # Map HF emotions → your clusters
    if label in ["joy", "surprise"]:
        return "happy"
    if label in ["sadness"]:
        return "sad"
    if label in ["anger"]:
        return "angry"
    if label in ["fear", "disgust"]:
        return "melancholic"
    if label in ["neutral"]:
        return "neutral"

    return "neutral"

In [ ]:
# Universal mood mapping (7-cluster or normal version)

mood_map = {
    "happy": "happy",
    "joy": "neutral",
    "love": "romantic",
    "excited": "energetic",
    "enthusiasm": "energetic",
    "amusement": "happy",
    "grateful": "calm",
    "thankful": "calm",
    "contentment": "calm",
    "relaxed": "calm",
    "calm": "calm",
    "relief": "calm",
    "nostalgia": "melancholic",
    "tender": "romantic",
    "sad": "sad",
    "grief": "sad",
    "loss": "sad",
    "pain": "melancholic",
    "fear": "sad",
    "scared": "sad",
    "disgusted": "sad",
    "angry": "angry",
    "anger": "angry",
    "annoyance": "angry",
    "anxious": "melancholic",
    "nervous": "melancholic",
    "confusion": "neutral",
    "awkwardness": "neutral",
    "bored": "melancholic",
    "craving": "energetic",
    "awe": "energetic",
    "adoration": "romantic",
    "energetic" : "energetic"
}

def normalize_mood_label(label):
    # If detected mood is already a cluster mood:
    if label in mood_to_cluster:
        return label

    # Convert to closest mood category
    return mood_map.get(label, "neutral")

In [ ]:
def recommend_songs_from_text_input(user_input, top_n=5):
    detected_mood = detect_mood_from_text(user_input)
    mapped_mood = normalize_mood_label(detected_mood)

    print(f"Detected mood: {detected_mood} -> using mood: {mapped_mood}\n")

    try:
        recs = recommend_by_mood(mapped_mood, top_n=top_n)

        if recs is not None and not recs.empty:
            print(f"Top {top_n} songs for mood '{mapped_mood}':")
            display(recs[['track_url','artist','song_name','mood','similarity']])
        else:
            print("No songs found for this mood.")
    except Exception as e:
        print("Error:", e)

In [ ]:
print(df.columns)
print(df['mood'].unique())
print(df['mood'].value_counts())

Index(['id', 'name', 'artist', 'track_url', 'valence', 'energy',
       'danceability', 'tempo', 'loudness', 'acousticness', 'speechiness',
       'instrumentalness', 'liveness', 'genre', 'cluster', 'mood'],
      dtype='object')
['calm' 'melancholic' 'angry' 'energetic' 'sad' 'happy' 'romantic']
mood
calm           36561
angry          26436
melancholic    22975
happy          11731
energetic       7789
sad             7392
romantic        1116
Name: count, dtype: int64


In [ ]:
recommend_songs_from_text_input("I am feeling depressed 😢", top_n=5)
recommend_songs_from_text_input("I am so energetic and calm 😊", top_n=5)
recommend_songs_from_text_input("Feeling angry", top_n=5)
recommend_songs_from_text_input("Feeling tired", top_n=5)
recommend_songs_from_text_input("I just want to relax 😴", top_n=5)

🎭 Detected mood: sad -> using mood: sad

🎶 Top 5 songs for mood 'sad':


,track_url,artist,song_name,mood,similarity
0,https://open.spotify.com/track/1qr2vHp7290W7KP...,Traditional;Magdalena Hoffmann,"O Tannenbaum, du trägst ein’ grünen Zweig (Arr...",sad,0.998269
1,https://open.spotify.com/track/7bDWIrRvL7KcZ5l...,Traditional;Magdalena Hoffmann,"O Tannenbaum, du trägst ein’ grünen Zweig (Arr...",sad,0.998269
2,https://open.spotify.com/track/7nEWerH9vQDQ1wN...,Con Alma,Halcyon,sad,0.997667
3,https://open.spotify.com/track/1dDMD5l9e7OtiZ2...,Wolfgang Amadeus Mozart;Ingrid Haebler;London ...,"Piano Concerto No. 23 in A Major, K. 488: II. ...",sad,0.997435
4,https://open.spotify.com/track/5H0E4n5CE6WFI10...,Stanton Lanier,Stunningly Beautiful,sad,0.996807


🎭 Detected mood: energetic -> using mood: energetic

🎶 Top 5 songs for mood 'energetic':


,track_url,artist,song_name,mood,similarity
0,https://open.spotify.com/track/719Dewg7OFXZEmA...,Los Fabulosos Cadillacs,Vasos Vacios - Versión Remasterizada (Live),energetic,0.993963
1,https://open.spotify.com/track/6GkUCsN1tEH24jo...,Jorge & Mateus,Do Brasil A Argentina (Tô Indo Te Buscar) - Ao...,energetic,0.992582
2,https://open.spotify.com/track/0nTXrWKh74sULHO...,Paulo Rogerio,Minha Casa e Eu - Ao Vivo,energetic,0.991835
3,https://open.spotify.com/track/6JMRXqVpDD1Xu2l...,O Rappa,Rodo cotidiano - Ao vivo,energetic,0.983604
4,https://open.spotify.com/track/6JMRXqVpDD1Xu2l...,O Rappa,Rodo cotidiano - Ao vivo,energetic,0.983604


🎭 Detected mood: angry -> using mood: angry

🎶 Top 5 songs for mood 'angry':


,track_url,artist,song_name,mood,similarity
0,https://open.spotify.com/track/6SHxM9QMWJIN3uY...,PXNDX,Claro Que No,angry,0.989459
1,https://open.spotify.com/track/6SHxM9QMWJIN3uY...,PXNDX,Claro Que No,angry,0.989459
2,https://open.spotify.com/track/1QcAm8PFwuZGa0r...,ROTTENGRAFFTY,「70cm四方の窓辺」,angry,0.986889
3,https://open.spotify.com/track/1TObetuMycXgZJR...,Seringai,Serigala Militia,angry,0.986548
4,https://open.spotify.com/track/5xMotDt8HMBgudG...,DEAD END,Embryo Burning,angry,0.985043


🎭 Detected mood: calm -> using mood: calm

🎶 Top 5 songs for mood 'calm':


,track_url,artist,song_name,mood,similarity
0,https://open.spotify.com/track/4CrvuZnTyzReLCc...,Manuel Carrasco,Yo Quiero Vivir,calm,0.985282
1,https://open.spotify.com/track/4CrvuZnTyzReLCc...,Manuel Carrasco,Yo Quiero Vivir,calm,0.985282
2,https://open.spotify.com/track/6Uj1ctrBOjOas8x...,Doja Cat,Woman,calm,0.984939
3,https://open.spotify.com/track/6Uj1ctrBOjOas8x...,Doja Cat,Woman,calm,0.984939
4,https://open.spotify.com/track/3BdMyHoBSt1jzGG...,Vybz Kartel,Beautiful Girl,calm,0.977883


🎭 Detected mood: calm -> using mood: calm

🎶 Top 5 songs for mood 'calm':


,track_url,artist,song_name,mood,similarity
0,https://open.spotify.com/track/4CrvuZnTyzReLCc...,Manuel Carrasco,Yo Quiero Vivir,calm,0.985282
1,https://open.spotify.com/track/4CrvuZnTyzReLCc...,Manuel Carrasco,Yo Quiero Vivir,calm,0.985282
2,https://open.spotify.com/track/6Uj1ctrBOjOas8x...,Doja Cat,Woman,calm,0.984939
3,https://open.spotify.com/track/6Uj1ctrBOjOas8x...,Doja Cat,Woman,calm,0.984939
4,https://open.spotify.com/track/3BdMyHoBSt1jzGG...,Vybz Kartel,Beautiful Girl,calm,0.977883


In [ ]:
recommend_songs_from_text_input("I am so happy and excited 😊", top_n=5)

🎭 Detected mood: energetic -> using mood: energetic

🎶 Top 5 songs for mood 'energetic':


,track_url,artist,song_name,mood,similarity
0,https://open.spotify.com/track/719Dewg7OFXZEmA...,Los Fabulosos Cadillacs,Vasos Vacios - Versión Remasterizada (Live),energetic,0.993963
1,https://open.spotify.com/track/6GkUCsN1tEH24jo...,Jorge & Mateus,Do Brasil A Argentina (Tô Indo Te Buscar) - Ao...,energetic,0.992582
2,https://open.spotify.com/track/0nTXrWKh74sULHO...,Paulo Rogerio,Minha Casa e Eu - Ao Vivo,energetic,0.991835
3,https://open.spotify.com/track/6JMRXqVpDD1Xu2l...,O Rappa,Rodo cotidiano - Ao vivo,energetic,0.983604
4,https://open.spotify.com/track/6JMRXqVpDD1Xu2l...,O Rappa,Rodo cotidiano - Ao vivo,energetic,0.983604


Save Artifacts

In [ ]:
df.to_csv("/content/processed_spotify_tracks.csv", index=False)
joblib.dump(feature_cols, "/content/feature_cols.joblib")
joblib.dump(id_to_idx, "/content/id_to_idx.joblib")
joblib.dump(kmeans, "/content/kmeans.joblib")
joblib.dump(scaler, "/content/scaler.joblib")

print("Saved processed CSV and model artifacts")

Saved processed CSV and model artifacts
